# Import backage

In [1]:
import sys
sys.path.append("../..")
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import random
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch import distributions
from torch.nn.parameter import Parameter
from train import train
from models.models_new import Renorm_Dynamic
from dynamic_models_sis_new import Simple_Spring_Model
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from datetime import datetime
t0 = datetime.now()


use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0') if use_cuda else torch.device('cpu')
device

device(type='cuda', index=0)

# Noise experiments

In [3]:
# nis+ CE macro
mul_batch_size = [0,5000,3000,1000]
sigma_list = [0.00001,0.0001,0.001,0.01,0.1,1]
experiments = len(sigma_list)
rou = -0.5
steps = 7
sz = 4
scale = 2
L = 1
mae2_w = 3
T_total = 30001


eis_macro_e = np.zeros([experiments,T_total//500+1])
losses_macro_e = np.zeros([experiments,T_total//500+1])
MAEs_mstep_macro_e = np.zeros([experiments,T_total//500+1])

In [ ]:
# nis+ micro
scale = 4
eis_micro_e = np.zeros([experiments,T_total//500+1])
losses_micro_e = np.zeros([experiments,T_total//500+1])
MAEs_mstep_micro_e = np.zeros([experiments,T_total//500+1])

for e in range(experiments):
    sigma = sigma_list[e]
    seed = 1 
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    spring = Simple_Spring_Model(device=device)
    test_data = spring.generate_multistep_sir(size_list=[500,500], steps=10, sigma=sigma, rou=rou,lam=1,miu=0.5,dt=0.01) #sir
    train_data = spring.generate_multistep_sir(size_list=mul_batch_size, steps=steps, sigma=sigma,rou=rou,lam=1,miu=0.5,dt=0.01)
    
    eis_nisp, term1s_nisp, term2s_nisp, losses_nisp, MAEs_mstep_nisp, net = train(train_data=train_data, test_data=test_data, 
                                                                    sigma=sigma, rou=rou, sz=sz, scale=scale, L=L, 
                                                                    mae2_w=mae2_w, T2=T_total,framework='nis+')
    eis_micro_e[e,:] = eis_nisp
    losses_micro_e[e,:] = losses_nisp
    MAEs_mstep_micro_e[e,:] = MAEs_mstep_nisp
    torch.save(net.state_dict(), 'mdl_data/micro noise = '+str(sigma)+'.mdl')

check point-------o_0-> 11:16:25.792079; lasting 0:00:02.561319 seconds
Epoch: 0
Train loss: 0.0890
dEI: 0.6528
term1: -0.1493
term2: 2.7607
Test multistep loss: 0.2998
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 11:16:30.180353; lasting 0:00:04.388274 seconds
check point-------o_0-> 11:16:45.700771; lasting 0:00:15.520418 seconds
Epoch: 500
Train loss: 0.0005
dEI: 3.1618
term1: 9.7955
term2: 2.8517
Test multistep loss: 0.0026
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 11:16:51.535782; lasting 0:00:05.835011 seconds
check point-------w_0-> 11:17:00.196957; lasting 0:00:08.661175 seconds
check point-------w_1-> 11:17:00.208596; lasting 0:00:00.011639 seconds
check point-------w_2-> 11:19:08.429087; lasting 0:02:08.220491 seconds
check point-------w_3-> 11:19:08.446309; lasting 0:00:00.01

In [ ]:
# nis+ macro
for e in range(experiments):
    sigma = sigma_list[e]
    seed = 1 
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    spring = Simple_Spring_Model(device=device)
    test_data = spring.generate_multistep_sir(size_list=[500,500], steps=10, sigma=sigma, rou=rou,lam=1,miu=0.5,dt=0.01) #sir
    train_data = spring.generate_multistep_sir(size_list=mul_batch_size, steps=steps, sigma=sigma,rou=rou,lam=1,miu=0.5,dt=0.01)
    
    eis_nisp, term1s_nisp, term2s_nisp, losses_nisp, MAEs_mstep_nisp, net = train(train_data=train_data, test_data=test_data, 
                                                                    sigma=sigma, rou=rou, sz=sz, scale=scale, L=L, 
                                                                    mae2_w=mae2_w, T2=T_total,framework='nis+')
    eis_macro_e[e,:] = eis_nisp
    losses_macro_e[e,:] = losses_nisp
    MAEs_mstep_macro_e[e,:] = MAEs_mstep_nisp
    torch.save(net.state_dict(), 'mdl_data/macro noise = '+str(sigma)+'.mdl')

In [ ]:
plt.figure(dpi=300)
color_label = ['#F4F1DE', '#DF7A5E', '#3C405B', '#82B29A', '#F2CC8E', '#A8DADC', '#457B9D', '#1D3557']
plt.plot(sigma_list, eis_macro_e[:,-1]-eis_micro_e[:,-1])
plt.legend()
plt.xlabel("epoch")
plt.ylabel(r'$\mathcal{J}$')
plt.show()

# Psi calculate

In [ ]:
import ei.entropy_estimators as ee

def psi(net,micro_data):
    ss,sps,ls,lps = micro_data
    predicts1, latent1, latentp1 = net.forward(ss)
    micro_mi = 0
    for dim in range(ss.size()[1]):
        micro_mi += ee.mi(ss[:,dim].cpu().numpy(),latenp1.cpu().numpy())
    macro_mi = ee.mi(latent1.cpu().numpy(), latentp1.cpu().numpy())
    p = macro_mi - micro_mi
    return p